In [ ]:
from google.colab import drive
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import spacy
from collections import Counter
import torch
import torchtext
import torch.nn as nn
import torch.optim as optim
import random

In [ ]:
!pip install torchtext==0.6.0 --quiet

from torchtext.data import Field, BucketIterator

In [ ]:
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# Read JSON data from file
with open('/gdrive/MyDrive/train_data1.json', 'r') as file:
    data = json.load(file)

In [ ]:
# Read JSON data from file
with open('/gdrive/MyDrive/test_data1_final.json', 'r') as file:
    test_data = json.load(file)

In [ ]:
import string

# Define the remove_punctuations function
def remove_punctuations(sentence):
    punctuations = list(string.punctuation)
    punctuations.append('।')
    punctuations.append('৷')
    punctuations.append('’')
    punctuations.append('‘')
    cleaned = ""
    for letter in sentence:
        if letter not in punctuations:
            cleaned += letter
    return cleaned


In [ ]:
import re

# Define a function to check if a sentence contains English words
def contains_english_words(sentence):
    return bool(re.search(r'[a-zA-Z]', sentence))

In [ ]:
# Process JSON data
source_sentences_train = []
target_sentences_train = []
len_kannada = []
len_english = []

id_train = []

for language_pair, language_data in data.items():
    if(language_pair == "English-Kannada"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"].lower()
              target = entry_data["target"].lower()
              source = remove_punctuations(source)
              target = remove_punctuations(target)
              if not contains_english_words(target):
                source_sentences_train.append(source)
                target_sentences_train.append(target)
                id_train.append(entry_id)
                len_kannada.append(len(source.split(' ')))
                len_english.append(len(target.split(' ')))

Language Pair: English-Kannada
  Data Type: Train


In [ ]:
# Calculate the maximum length of kannada and English sentences
max_length_kannada = max(len_kannada)
max_length_english = max(len_english)

print(f"Maximum Length of kannada Sentence: {max_length_kannada} words")
print(f"Maximum Length of English Sentence: {max_length_english} words")

Maximum Length of kannada Sentence: 100 words
Maximum Length of English Sentence: 78 words


In [ ]:
len(target_sentences_train)

46114

In [ ]:
test_source = []
valid_id = []
len_test=[]


for language_pair, language_data in test_data.items():
    if(language_pair == "English-Kannada"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"].lower()
              source = remove_punctuations(source)
              test_source.append(source)
              valid_id.append(entry_id)
              len_test.append(len(source.split(' ')))

Language Pair: English-Kannada
  Data Type: Test


In [ ]:
test_source[1]

'in 1997 he first played the supporting role of vikram dixit in the film ghulamemustafa starring nana patekar and raveena tandon'

In [ ]:
print(max(len_test))

75


In [ ]:
nlp_en = spacy.load('en_core_web_sm')
# Tokenize English sentences
tokenized_source_sentences = []
for sentence in source_sentences_train:
    doc = nlp_en(sentence)
    tokens = [token.text for token in doc]
    tokenized_source_sentences.append(tokens)

In [ ]:
tokenized_test_source = []
for sentence in test_source:
    doc = nlp_en(sentence)
    tokens = [token.text for token in doc]
    tokenized_test_source.append(tokens)

In [ ]:
!pip install nltk

In [ ]:
pip install indic-nlp-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 12.2 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download("indic_nltk")

from indicnlp.tokenize import indic_tokenize


# Tokenize kannada sentences
tokenized_target_sentences = []
for sentence in target_sentences_train:
    tokens = indic_tokenize.trivial_tokenize(sentence)
    tokenized_target_sentences.append(tokens)

[nltk_data] Error loading indic_nltk: Package 'indic_nltk' not found
[nltk_data]     in index


In [ ]:
from torchtext.data import Dataset, Example, Field

def tokenize_english(text):
    return text

def tokenize_kannada(text):
    return text

class CustomTranslationDataset(Dataset):
    def __init__(self, source_sentences, target_sentences, source_field, target_field):
        fields = [('source', source_field), ('target', target_field)]
        examples = []
        for src, tgt in zip(source_sentences, target_sentences):
            examples.append(Example.fromlist([src, tgt], fields))
        super().__init__(examples, fields)

english = Field(tokenize=tokenize_english,
                lower=True,
                init_token="<sos>",
                eos_token="<eos>")

kannada = Field(tokenize=tokenize_kannada,
              lower=True,
              init_token="<sos>",
              eos_token="<eos>")

train_dataset = CustomTranslationDataset(tokenized_source_sentences, tokenized_target_sentences, english, kannada)


english.build_vocab(train_dataset,max_size=30000, min_freq=3)
kannada.build_vocab(train_dataset,max_size=30000, min_freq=3)



In [ ]:
print(len(english.vocab), len(kannada.vocab))

14225 22382


In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32

train_iterator = BucketIterator(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    sort_within_batch=True,
    sort_key=lambda x: len(x.source),
    device=device
)


In [ ]:
count = 0
for data in train_iterator:
  if count < 1 :
    print("Shapes", data.source.shape, data.target.shape)
    print()
    print("English - ",*data.source, " Length - ", len(data.source))
    print()
    print("kannada - ",*data.target, " Length - ", len(data.target))
    temp_eng = data.source
    temp_kannada = data.target
    count += 1


Shapes torch.Size([20, 32]) torch.Size([22, 32])

English -  tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([ 208,  117,    4,    4,  508,   25,  184,    6,  143,   64,  176,  421,
           6,   43,   61, 5384,   12,    4,  554,    4, 4089, 1524,    6, 7602,
          38,   18,   16,    0,  281, 1297,   37, 4073], device='cuda:0') tensor([   9,  775,  633,   76,    8,    9,    4,   11, 2485,  106,    4, 4202,
         188, 1391,    5,  296,   50, 2131,   19,  217,  562,   93,    4,   54,
           4,  100, 2470,    0,   11,  711,   74,    9], device='cuda:0') tensor([ 304,  242,   23,    5, 1230, 1908, 3557, 1569,   53,   13,  114, 5481,
         262,   53,   98,  550,   13,    5,  225,    5,    4, 1463,    0,    0,
        2868, 2236, 4511,   84, 3364,  647,   32,   34], device='cuda:0') tensor([  10,    9,  213,  838,    5,    5,  190, 4548,   29,    8, 1257,   10,
           5,   36,   63,  

In [ ]:
temp_eng_idx = (temp_eng).cpu().detach().numpy()
temp_kannada_idx = (temp_kannada).cpu().detach().numpy()

In [ ]:
e = list(english.vocab.__dict__.values())
word_2_idx = dict(e[3])
idx_2_word = {}
for k,v in word_2_idx.items():
  idx_2_word[v] = k


In [ ]:
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderLSTM, self).__init__()

    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    self.embedding = nn.Embedding(input_size, embedding_size)

    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

  def forward(self, x):

    embedding = self.dropout(self.embedding(x))
    outputs, (hidden_state, cell_state) = self.LSTM(embedding)

    return hidden_state, cell_state


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size_encoder = len(english.vocab)
encoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
encoder_dropout = 0.5

encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)

EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(14225, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
)


In [ ]:
class DecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
    super(DecoderLSTM, self).__init__()
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers
    self.output_size = output_size

    # Regularization parameter
    self.dropout = nn.Dropout(p)

    self.embedding = nn.Embedding(input_size, embedding_size)

    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x, hidden_state, cell_state):

    x = x.unsqueeze(0)
    embedding = self.dropout(self.embedding(x))

    outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))


    predictions = self.fc(outputs)

    predictions = predictions.squeeze(0)

    return predictions, hidden_state, cell_state

In [ ]:
input_size_decoder = len(kannada.vocab)
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
decoder_dropout = 0.5
output_size = len(kannada.vocab)

decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, decoder_dropout, output_size).to(device)
print(decoder_lstm)

DecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(22382, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=1024, out_features=22382, bias=True)
)


In [ ]:

class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM

  def forward(self, source, target, tfr=0.5):

    batch_size = source.shape[1]

    target_len = target.shape[0]
    target_vocab_size = len(kannada.vocab)


    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

    hidden_state, cell_state = self.Encoder_LSTM(source)

    x = target[0]

    for i in range(1, target_len):

      output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
      outputs[i] = output
      best_guess = output.argmax(1)
      x = target[i] if random.random() < tfr else best_guess

    return outputs

In [ ]:

# Hyperparameters

learning_rate = 0.001

step = 0

model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = kannada.vocab.stoi[""]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [ ]:
model

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(14225, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  )
  (Decoder_LSTM): DecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(22382, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=22382, bias=True)
  )
)

In [ ]:
def pred_trans(model, tokens, english, kannada, device, max_length=100):
    tokens.insert(0, english.init_token)
    tokens.append(english.eos_token)
    text_to_indices = [english.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)

    outputs = [kannada.vocab.stoi[""]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == kannada.vocab.stoi[""]:
            break

    translated_sentence = [kannada.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:]

In [ ]:
import time
from tqdm import tqdm
epoch_loss = 0.0
num_epochs = 10

sentence1 = "the person with sight should stand in front of the blind person"


for epoch in range(num_epochs):
  start_time = time.time()
  print("Epoch - {} / {}".format(epoch+1, num_epochs))
  model.eval()
  doc = nlp_en(sentence1)
  tok = [token.text for token in doc]
  translated_sentence1 = pred_trans(model, tok, english, kannada, device, max_length=100)
  print(f"Translated example sentence 1: \n {translated_sentence1}"

  model.train(True)
  for batch_idx, batch in tqdm(enumerate(train_iterator), total=len(train_iterator)):
    input = batch.source.to(device)
    target = batch.target.to(device)

    # Pass the input and target for model's forward method
    output = model(input, target)
    output = output[1:].reshape(-1, output.shape[2])
    target = target[1:].reshape(-1)

    # Clear the accumulating gradients
    optimizer.zero_grad()

    # Calculate the loss value for every epoch
    loss = criterion(output, target)

    # Calculate the gradients for weights & biases using back-propagation
    loss.backward()

    # Clip the gradient value is it exceeds > 1
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    optimizer.step()
    step += 1
    epoch_loss += loss.item()

  # Calculate the time taken for the epoch
  end_time = time.time()
  epoch_time = end_time - start_time
  print(f"Time taken for epoch {epoch + 1}: {epoch_time:.2f} seconds")

  if epoch_loss < best_loss:
    best_loss = epoch_loss
    best_epoch = epoch
    if ((epoch - best_epoch) >= 10):
      print("no improvement in 10 epochs, break")
      break
  print("Epoch_Loss - {}".format(loss.item()))
  print()

print(epoch_loss / len(train_iterator))


Epoch - 1 / 10
Translated example sentence 1: 
 ['ವಾಘಾ', 'ಎಲ್ಲಿಗೆ', 'ವಾಘಾ', 'ಸೆಂಟಿಗ್ರೇಡ್', 'ಸೆಂಟಿಗ್ರೇಡ್', 'ಹಿಡಿದುಕೊಂಡು', 'ವ್ಯವಸ್ಥೆಗಳ', 'ಅಕ್ಷರಶಃ', 'ಅಕ್ಷರಶಃ', 'ಹೊಸದಾಗಿ', 'ಹಿಮೋಗ್ಲೋಬಿನ್', 'ಖಂಡೋಲಿ', 'ದೊರಕಿದೆ', 'ದೊರಕಿದೆ', 'ದೊರಕಿದೆ', 'ದೊರಕಿದೆ', 'ಬೀರಬಹುದು', 'ನಾಲ್ಕನೇ', 'ಅಧಿಸೂಚನೆ', 'ಅಧಿಸೂಚನೆ', 'ವಸ್ತುಗಳಲ್ಲಿ', 'ತಯಾರಿಸಲು', 'ಬರುತ್ತಿದ್ದಂತೆಯೇ', 'ಎಲ್ಲವೂ', 'ಎಲ್ಲವೂ', 'ಹೊರಬರುತ್ತವೆ', 'ಸಮಗ್ರತೆಯ', 'ಸಮಗ್ರತೆಯ', 'ಪಾಕಿಸ್ತಾನದ', 'ಸೂರ್ಯೋದಯ', 'ಪಾಕಿಸ್ತಾನದ', 'ಕೆಂಪಾಗುವುದು', 'ಮೀಟರ್\u200cಗಳಷ್ಟು', 'ಮಲಗುವುದು', 'ದೂರಮಾಡಲು', 'ದೂರಮಾಡಲು', 'ದೂರಮಾಡಲು', 'ನಾಲ್ಕೂವರೆ', 'ಆಯೋಜಿಸಿದರು', 'ಕೆಂಪಾಗುವುದು', 'ಕೆಂಪಾಗುವುದು', 'ಟ್ಯೂಲಿಪ್', 'ಜನ್ಮಭೂಮಿ', 'ಇದರಲ್ಲಿ', 'ಸೆಂಟಿಗ್ರೇಡ್', 'ವ್ಯವಸ್ಥೆಗಳ', 'ವ್ಯವಸ್ಥೆಗಳ', 'ಹೊಸದಾಗಿ', 'ನೋಡುವುದರ', 'ನೋಡುವುದರ', 'ಕೈಬಿಡಲಾಯಿತು', 'ಕೈಬಿಡಲಾಯಿತು', 'ನಾಲ್ಕೂವರೆ', 'ಕೈಬಿಡಲಾಯಿತು', 'ಕೈಬಿಡಲಾಯಿತು', 'ಸರಾಸರಿಯಲ್ಲಿ', 'ಗುಣಮಟ್ಟದಲ್ಲಿ', 'ಗುಣಮಟ್ಟದಲ್ಲಿ', 'ಮುಖಂಡರು', 'ಕೈಬಿಡಲಾಯಿತು', 'ಕೈಬಿಡಲಾಯಿತು', 'ಕೈಬಿಡಲಾಯಿತು', 'ಸರಾಸರಿಯಲ್ಲಿ', 'ಗುಣಮಟ್ಟದಲ್ಲಿ', 'ಸರಾಸರಿಯಲ್ಲಿ', 'ಕಾಯಿದೆ', 'ಕಾಯಿದೆ', 'ಇಬ್ಬರಲ್ಲಿಯೂ', 'ಇಬ್ಬರಲ್ಲಿಯೂ', 'ಕೈಬಿಡಲಾಯಿತು', 'ಕೈಬಿಡಲಾಯಿತು', 'ಕೈಬಿಡಲಾಯಿತು

100%|██████████| 1442/1442 [04:26<00:00,  5.40it/s]


Time taken for epoch 1: 268.24 seconds
Epoch_Loss - 5.276762962341309

Epoch - 2 / 10
Translated example sentence 1: 
 ['ಈ', 'ರಾಷ್ಟ್ರೀಯ', 'ಭಾರತದ', 'ಒಂದು', 'ಒಂದು', '<eos>', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 

100%|██████████| 1442/1442 [04:26<00:00,  5.41it/s]


Time taken for epoch 2: 267.07 seconds
Epoch_Loss - 5.617408752441406

Epoch - 3 / 10
Translated example sentence 1: 
 ['ಈ', 'ಸಮಯದಲ್ಲಿ', 'ಹೆಚ್ಚು', 'ಹೆಚ್ಚು', 'ಹೆಚ್ಚು', '<eos>', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad

...

Time taken for epoch 8: 257.04 seconds
Epoch_Loss - 1.2380298376083374

Epoch - 9 / 10
Translated example sentence 1: 
 ['ಸಾಮಾನ್ಯ', 'ಹಾಲಿನಿಂದ', 'ನರಳುವ', 'ಆದಾಗ', 'ಕಣ್ಣಿನ', 'ಗಮನ', 'ಕೊಡಬೇಕು', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '

100%|██████████| 1442/1442 [04:21<00:00,  5.52it/s]


Time taken for epoch 9: 261.71 seconds
Epoch_Loss - 0.9263932108879089

Epoch - 10 / 10
Translated example sentence 1: 
 ['ತಾಯಿಯ', 'ವ್ಯಕ್ತಿಯು', 'ನರಳುವ', 'ಚಳಿಯಲ್ಲಿ', 'ಚಳಿಯಲ್ಲಿ', '<eos>', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pa

100%|██████████| 1442/1442 [04:26<00:00,  5.41it/s]

Time taken for epoch 10: 267.41 seconds
Epoch_Loss - 2.911987781524658

35.04805422023497


In [ ]:
s = test_source[10]
translation = translate_sentence(model, s, english, kannada, device, max_length=128)
cleaned_tokens = [token for token in translation if token not in ['<eos>', '<pad>']]
readable_translation = ' '.join(cleaned_tokens)
print(readable_translation)

ಕೆಂಪು ಮತ್ತು ಮತ್ತು ಹಳದಿ ಪ್ರಮಾಣದಲ್ಲಿ ಕಂಡುಬರುತ್ತದೆ


In [ ]:
kannada_pred = []

for tokens in tokenized_test_source:
  translation = pred_trans(model, tokens, english, kannada, device, max_length=100)
  cleaned_tokens = [token for token in translation if token not in ['<eos>', '<pad>','।','৷']]
  pred = ' '.join(cleaned_tokens)
  print(pred)
  kannada_pred.append(pred)


Streaming output truncated to the last 5000 lines.
ವಾಸ್ತವವಾಗಿ ಈ ಈ ಮೊದಲು ಇದರ ದಾಟುವುದು ಇದರ ಇರುತ್ತದೆ
ಬರಿಯ ಹೊಸ ಸಂಚಿಕೆಯನ್ನು ಪ್ಲೇ ಮಾಡಿ
ಬಾಲ್ಯದ ಚಿತ್ರದ ತನ್ನ ಪಾದಾರ್ಪಣೆ ಅವರು ಅವರು ಅವರು “ಮೂ” ಗೆಳೆಯ
ಸಮುದ್ರ ಮಟ್ಟದಿಂದ ಎತ್ತರದ ಸುಮಾರು ಅಡಿ ಅಡಿ ಎತ್ತರ ಎತ್ತರ ಅಡಿ ಎತ್ತರ ಮತ್ತು ಮತ್ತು ಅಡಿ ಮೀಟರ್ ಇದೆ
ಜಾನ್ಸ್ ಆಡಿಯೊಬುಕ್ ಆಡಿಯೊಬುಕ್ ಅನ್ನು ಪ್ಲೇ ಪ್ರಾರಂಭಿಸಿ
ಬ್ರಹ್ಮನು ರಾಜಸ್ಥಾನ ಮತ್ತು ಪಶ್ಚಿಮದಲ್ಲಿ ಪೂರ್ವದಲ್ಲಿ ಇದು ಇದು ರಾಜ್ಯದ ರಾಜ್ಯದ ಮತ್ತು ಮತ್ತು ಮತ್ತು ಮತ್ತು ರಾಜ್ಯದ ರಾಜ್ಯದ ರಾಜ್ಯವನ್ನು
ಆದ್ದರಿಂದ ಈ ನಟನಾ ಮುಹಮ್ಮದ್ ಮುಹಮ್ಮದ್ ಅವರ ಪ್ರಸರಣ ಅವರ ಚೆಂಡನ್ನು ಅವರ ಅವರ ಅನುಮೋದನೆ
ನಂತರ ನಿಮಗೆ ನಿಮ್ಮ ನಿಮ್ಮ ನಿಮ್ಮ ನಿಮ್ಮ ನೋಡಿದಾಗ
ನಂಗೆ ಮೈಲಿಯಲ್ಲಿ ಸಮಯ ಒಳ್ಳೆಯದು
ಮೊಡವೆ ಒಂದು ವ್ಯಕ್ತಿಯ ವ್ಯಕ್ತಿಯ ರಚನಾತ್ಮಕ ರಚನಾತ್ಮಕ ಒಂದು ಒಂದು ಒಂದು
ಕೈಕಾಲುಗಳಲ್ಲಿ ಕುಳಿತರೆ ಮತ್ತು ಮತ್ತು ಪಿತ್ತಜನಕಾಂಗ ಇರಿ
ನಾವು ನಾವು ಜೊತೆಗಾರರು ನಾವು ತಲುಪಿದೆವು
ಒಂದು ಸುಮಾರು ಒಂದು ಪ್ರವಾಸದ ಬಗ್ಗೆ ವಿಧಗಳನ್ನು
ಹಿಪ್ಪುನೇರಳೆಯ ಮತ್ತು ಮತ್ತು ಮತ್ತು ಮತ್ತು ಸುದ್ದಿಗಾಗಿ ಹಿಪ್ಪುನೇರಳೆ ಸುದ್ದಿಗಾಗಿ
ಯಾವುದೇ ಗಂಟೆಯಲ್ಲಿ ಯಾವುದೇ ರೀತಿಯ
ಅವನು 12ನೆಯ ಲಿಪಿಯನ್ನು ದೊರೆತ ಹೇಳಿದರು ಹೇಳಿದರು
ಈ ಸರ್ಕಾರವು ವಿದೇಶಿ ಯುದ್ಧದ ತೆಗೆದುಕೊಳ್ಳಲಾದ ತೆಗೆದುಕೊಳ್ಳಲಾದ ಈ ಈ ಮತ್ತು ಮತ್ತು ಮತ್ತು ಮತ್ತು ಮ

In [ ]:
import csv

# Create a list of dictionaries where each dictionary represents a row
data = [{'valid_id': valid_id[i], 'kannada_pred': kannada_pred[i]} for i in range(len(valid_id))]

# Specify the CSV file path
csv_file_path = '/gdrive/MyDrive/kannada3.csv'

# Define the column names
fields = ['valid_id', 'kannada_pred']

# Write the data to the CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    writer.writerows(data)

print(f'Saved predictions to {csv_file_path}')

Saved predictions to /gdrive/MyDrive/kannada3.csv


In [ ]:
print(len(kannada_pred))
print(len(test_source))

13371
13371
